# EDA: Digging into the Frequency and Severity of the dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
motor_df = pd.read_csv('../data/input/exp/Motor_vehicle_insurance_data.csv', delimiter=";")
claims_df = pd.read_csv('../data/input/exp/sample_type_claim.csv', delimiter=';')

### Claims Frequency Distribution - 2 methods documented

- Number: Categorical group for number of claims ranging from 0 to 25 (maximum number of claims made by 1 policyholder)
- Policies: Number of unique policies per group
- Claims: Total number of claims per group
- Proportion: Ratio of each group's policies to total number of policies.

#### Method 1 - Readable

In [ ]:
# combined  =  pd.merge(motor_df, claims_df, on='ID', how='left')
# bins = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
# labels = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24+']

In [4]:
## Get the maximum claims by an ID (an ID is unique value for a policyholder and product)
# maximum_claim  = (
#     motor_df
#     .groupby(['ID'])
#     .agg({'ID':'nunique', 'N_claims_year': 'sum'})
#     .max()
# )

def assign_claims_bin(claims_value):
    if claims_value <= 25:
        return str(claims_value)
    else:
        return '25+'

##### assign bin to each row of the motor_df
motor_df['claim_bin'] = motor_df['N_claims_year'].apply(assign_claims_bin)

In [5]:
motor_df['product_id'] = motor_df['ID'].astype(str) + motor_df['Date_last_renewal'].astype(str) + motor_df['Date_next_renewal'].astype(str)

In [10]:
grouping = (
    motor_df
    .groupby('claim_bin', observed=True)
    .agg({'product_id': 'nunique', 'N_claims_year': 'sum'})
    .rename(columns={'product_id': 'Policies', 'N_claims_year': 'Claims'})
    .reset_index()
    .rename(columns={'claims_bin': 'claim_group'})
)
##Calculate grouping proportion
grouping['Proportion'] = (grouping['Policies'] / grouping['Policies'].sum()).round(4)

## Summing and appending as a row
length_of_grouping = len(grouping)
grouping.loc[length_of_grouping] = ['Sum', grouping['Policies'].sum(), grouping['Claims'].sum(), grouping['Proportion'].sum()]

##Transpose df
grouping_transposed  = grouping.T

##clean out the columns
grouping_transposed.columns = [''] * len(grouping_transposed.columns)
grouping_transposed

,,,,,,,,,,,,,,,,,,,,,,,
claim_bin,0,1,10,11,12,13,14,15,16,17,18,19,2,21,25,3,4,5,6,7,8,9,Sum
Policies,85909,9539,61,24,18,18,10,8,2,3,2,1,4961,1,1,2435,1190,609,318,227,136,82,105555
Claims,0,9539,610,264,216,234,140,120,32,51,36,19,9922,21,25,7305,4760,3045,1908,1589,1088,738,41662
Proportion,0.8139,0.0904,0.0006,0.0002,0.0002,0.0002,0.0001,0.0001,0.0,0.0,0.0,0.0,0.047,0.0,0.0,0.0231,0.0113,0.0058,0.003,0.0022,0.0013,0.0008,1.0002


#### Method 2 - Chained

In [ ]:
combined = pd.merge(motor_df, sample_df, on='ID', how='left')
bins = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
labels = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24+']


def add_sum_row(df):
    df.loc[len(df)] = ['Sum', df['Policies'].sum(), df['Claims'].sum(), df['Proportion'].sum()]
    return df

claims_frequency_summary = (
    combined.groupby(pd.cut(combined['N_claims_year'], bins=bins, labels=labels), observed=True)
    .agg({'ID': 'nunique', 'N_claims_year': 'sum'})
    .rename(columns={'ID': 'Policies', 'N_claims_year': 'Claims'})
    .reset_index()
    .rename(columns={'N_claims_year': 'Number'})
    .assign(Proportion=lambda df: (df['Policies'] / df['Policies'].sum()).round(4))
    .pipe(add_sum_row)
    .T
    .pipe(lambda df: df.rename_axis('', axis=1))
    .pipe(lambda df: df.set_axis([''] * len(df.columns), axis=1))
)
claims_frequency_summary

### Claims Severity Distribution - TBI

- Average severity (loss cost) made by each policyholder that initiated claims expressed in logarithimic scale
- Visual inspection and confirmation of numbers required

In [ ]:
claims_summary  =  (
    sample_df[['Cost_claims_year']].drop_duplicates(keep='first') ### Sample df already aggregated the loss cost to each policyholder just need to drop duplicates
    .describe()
    .drop('count')
    .T
    .reset_index(drop=True)
)
claims_summary

In [ ]:
claims  = sample_df[['Cost_claims_year']].drop_duplicates(keep='first')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

fig, ax1.hist(claims['Cost_claims_year'])
ax1.set_xlabel("Average Claims")
ax1.set_title("")

# Histogram of log(yAvg)
ax2.hist(np.log(claims['Cost_claims_year']))
ax2.set_xlabel("Logarithmic Average Claims")
ax2.set_title("")

plt.tight_layout()
plt.show()